<a href="https://colab.research.google.com/github/christianwarmuth/openhpi-kipraxis/blob/main/Woche%203/3_5_Sentiment_Analyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Installieren aller Pakete

In [ ]:
# Hier die Kaggle Credentials einfügen (ohne Anführungszeichen)

%env KAGGLE_USERNAME=openhpi
%env KAGGLE_KEY=das_ist_der_key

In [ ]:
!pip install skorch

In [ ]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from skorch import NeuralNetClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
from collections import Counter
import re
from bs4 import BeautifulSoup
import yaml
import os

from wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import spacy
!python -m spacy download en_core_web_sm

import torch
from torch import nn


In [ ]:
class NeuralNetModule(nn.Module):
    def __init__(self, num_inputs, num_units=20, nonlin=nn.ReLU()):
        super(NeuralNetModule, self).__init__()

        self.nonlin = nonlin
        self.dense0 = nn.Linear(num_inputs, num_units)
        self.dropout = nn.Dropout(0.2)
        self.dense1 = nn.Linear(num_units, num_units)
        self.output = nn.Linear(num_units, 2)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = self.nonlin(self.dense1(X))
        X = self.softmax(self.output(X))
        return X


# 3.5 Sentiment Analyse

<img width=70% src="https://raw.githubusercontent.com/christianwarmuth/openhpi-kipraxis/main/images/cover_sentiment.jpeg">

### Was wir erreichen wollen

In diesem Anwendungsfall wollen wir Filmbewertungen in positive und negative Bewertungen unterteilen. Dafür liegen uns eine Vielzahl gelabelter Trainingsdaten in englischer Sprache vor.

Mit einem Modell wollen wir in der Lage sein, für neue Kommentare automatisiert die Stimmung analysieren zu können. 

## Download Dataset 

### Manuell
via https://www.kaggle.com/columbine/imdb-dataset-sentiment-analysis-in-csv-format

### Via API

Hinzufügen der kaggle.json
Speichern als ~/.kaggle/kaggle.json auf Linux, OSX, oder andere UNIX-based Betriebssysteme und unter C:\Users<Windows-username>.kaggle\kaggle.json auf Windows

Siehe https://www.kaggle.com/docs/api oder https://github.com/Kaggle/kaggle-api
        
Beispiel:
~/kaggle/kaggle.json

{"username":"openHPI","key":"das_ist_der_key"}

In [ ]:
!pip3 install kaggle
!kaggle datasets download -d columbine/imdb-dataset-sentiment-analysis-in-csv-format

In [ ]:
import zipfile
with zipfile.ZipFile("imdb-dataset-sentiment-analysis-in-csv-format.zip", 'r') as zip_ref:
    zip_ref.extractall("")

import os  
os.rename('Train.csv','sentiment.csv') 

In [ ]:
FILE_PATH = "sentiment.csv"

### Daten vorbereiten

Wir können nun unsere Daten laden und mit der Vorbereitung beginnen.

In [ ]:
df = pd.read_csv(FILE_PATH, nrows=10_000)
df.head() # label 1 == positive; label 0 == negative

Schauen wir uns zunächst einmal die Datenverteilung unserer Zielvariable an. Die Zielvariable ist kategorisch in diesem Fall:

In [ ]:
df["label"].value_counts(normalize=True)

Wir sehen: unsere Daten sind nahezu perfekt gleichverteilt (50% positiv, 50% negativ). Das ist prinzipiell eine gute Nachricht für unser Trainingsvorhaben (je unbalancierter Daten sind, desto komplexer wird das Training i.d.R.).

Werfen wir einen Blick auf das erste Beispiel, um ein Gefühl für die Datenbeschaffenheit zu erlangen:

In [ ]:
df["text"].iloc[0]

Wir stellen fest:
- Eine einzelne Bewertung kann recht lange sein. Das bedeutet für uns, dass Beispiele sowohl mehr Informationen, aber auch mehr "Rauschen" (für uns unwichtige Informationen) beinhalten können.
- Es handelt sich um Webdaten, da HTML-Tags gesetzt sind. Wir sollten diese korrekt verarbeiten.
- 's werden mit \\'s kodiert (was wiederum auf den Rohtext zurückzuführen ist)
- Wir sollten die Wörter im Text zu Kleinbuchstaben konvertieren, um die Anzahl möglicher Wort-Variationen zu reduzieren. 

In [ ]:
# preprocessing
df["text"] = df["text"].apply(lambda x: x.lower())
df["text"] = df["text"].apply(lambda x: x.replace("\'", ""))
df["text"] = df["text"].apply(lambda x: BeautifulSoup(x).text)

Schauen wir uns die gleiche Bewertung erneut an:

In [ ]:
df["text"].iloc[0]

Das sieht schon deutlich besser aus. Fangen wir nun an, unsere Daten zu splitten. Wir nutzen hier der Einfachheit halber einen ganz simplen Split mit Standardeinstellungen (wir könnten auch andere Verfahren einsetzen).

Wir teilen unsere Daten dabei in Trainings-, Validierungs- und Testsplit. Mit dem ersten trainieren wir die Modelle, mit dem zweiten suchen wir unser bestes Modell heraus, mit dem letzten validieren wir die Ergebnisse.

In [ ]:
train_df, test_valid_df = train_test_split(df)
test_df, valid_df = train_test_split(test_valid_df)

Bevor wir unsere KI-Pipeline bauen, schauen wir uns die Daten erneut an. Sicherheitshalber duplizieren wir unseren Trainingsdatensatz dafür. Wir schauen uns zunächst die Textlängen an:

In [ ]:
analysis_df = train_df.copy()
analysis_df["text_length"] = analysis_df["text"].apply(len)

Nun können wir uns die Verteilungen einfach je Klasse ("positiv" und "negativ") visualisieren lassen:

In [ ]:
sns.histplot(data=analysis_df, x="text_length", hue="label", element="step");

Wir erkennen keinen großen Unterschied zwischen den beiden Klassen in der Textlänge (d.h. sowohl positive als auch negative Bewertungen können sehr ausführlich sein).

Schauen wir uns den Textkorpus nochmal genauer an:

In [ ]:
text_corpus = " ".join(analysis_df["text"])

Bauen wir uns nun eine Hilfsfunktion, mit der wir uns einfach die *n* häufigsten Wörter des Korpus anschauen können. Als Korpus bezeichnet man eine Sammlung von Texten. 

In [ ]:
def plot_most_common_words(text_corpus, n):
    counter = Counter(text_corpus.split())
    rank, words, occurences = [], [], []
    for idx, (word, occurence) in enumerate(counter.most_common(n=n)):
        rank.append(idx)
        words.append(word)
        occurences.append(occurence)
        
    fig, ax = plt.subplots()
    ax.scatter(rank, occurences, s=0.01)
    for idx, word in enumerate(words):
        ax.annotate(word, (rank[idx], occurences[idx]))
    plt.title("Zipf's law")
    plt.ylabel("Occurences")
    plt.xlabel("Rank");


Wir können nun in der Verteilung das sogenannte Zipf's Law erkennen: Jedes Wort tritt ungefähr invers proportional zu seinem Rang auf, d.h. das häufigste Wort doppelt so häufig wie das zweithäufigste. Solche Füllwörter machen einen Großteil unserer Daten aus. Das ist ein ganz bekanntes Phönomen in natürlichen Sprachen.

In [ ]:
plot_most_common_words(text_corpus, 15)

Wir können diese Wörter entfernen, da sie inhaltlich keinen Mehrwert bringen. Als ersten Schritt erstellen wir ein Embedding für einzelne Wörter. Schauen wir uns die Verteilung einmal an, wenn wir die typischen Füllwörter entfernen:

In [ ]:
pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
text_corpus_without_stopwords = pattern.sub('', text_corpus)

Wir erzeugen die gleiche Visualisierung:

In [ ]:
plot_most_common_words(text_corpus_without_stopwords, 15)

Wir können schon deutlich erkennen, dass es sich um einen Datenbestand zu Filmbewertungen handelt. Wir können uns das auch als Wordcloud ("Wortwolke") anschauen. Wir schreiben uns hierfür eine Funktion:

In [ ]:
def draw_word_cloud(text_corpus):
    word_cloud = WordCloud(
        collocations = False, background_color = 'white'
    ).generate(text_corpus)
    plt.imshow(word_cloud, interpolation='bilinear')
    plt.axis("off");

Damit können wir die Worthäufigkeiten als Wordcloud darstellen lassen:

In [ ]:
draw_word_cloud(text_corpus_without_stopwords)

*Hinweis*:

An sich liefert eine Wordcloud vom Informationsgehalt keinen Mehrwert gegenüber einer einfachen Darstellung anhand eines Scatter Plots. Tatsächlich ist es für uns sogar schwieriger zuzuordnen, welches Wort am häufigsten vorkommt (`movie` oder `film`?). Außerdem müssen wir häufig unsere Leserichtung anpassen. Vorteilhaft ist lediglich die platzsparende Art der Darstellung.

Word Clouds werden gerne bei Textvisualisierungen verwendet - durchaus auch, weil es schlichtweg "gut" aussieht. 

Wir können uns diese Darstellung auch einmal je Klasse ("positiv" und "negativ") anzeigen; vielleicht erkennen wir klassenspezifische Wörter:

In [ ]:
positive_corpus = " ".join(analysis_df["text"].loc[analysis_df["label"] == 1])
negative_corpus = " ".join(analysis_df["text"].loc[analysis_df["label"] == 0])

Schauen wir uns zunächst die positiven Filmbewertungen in einer Wordcloud an:

In [ ]:
draw_word_cloud(positive_corpus)

Und anschließend die negativen Bewertungen:

In [ ]:
draw_word_cloud(negative_corpus)

Wir stellen fest, dass es Wörter gibt, die wir intuitiv vielleicht einer Klasse zugeordnet hätten (`good` -> positiv), aber auch in der gegenteiligen Klasse auftreten. Entstehen können solche Szenarien dann, wenn Wörter im Kontext des Satzes ihre Bedeutung ändern: `This film was really good.` vs. `In my opinion, this movie was not good at all.`

Wir könnten hier noch tiefer in eine Analyse gehen und versuchen, mehr über unsere Daten zu verstehen (z.B. wie oft treten verneinte Sätze auf? Was sind klassentrennde Wörter?). An dieser Stelle werden wir aber für dieses Projekt die explorative Analyse beenden und mit der KI-Pipeline beginnen.

# 3.5 Sentiment Analyse

<img width=70% src="https://raw.githubusercontent.com/christianwarmuth/openhpi-kipraxis/main/images/cover_sentiment.jpeg">

### Lemmatisierung

Wir können unsere Texte nun so anpassen, dass wir Wörter auf ihren Wortstamm reduzieren. Dafür eignet sich die Lemmatisierung.

In [ ]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(
    "lemmatization is the process of grouping together the inflected forms of a word."
)
for token in doc:
    print(token, token.lemma_)

Wir bauen uns hier wieder eine Funktion für:

In [ ]:
def enrich_lemmatized_text(df):
    df = df.copy()
    df["lemmatized_text"] = df["text"].apply(
        lambda x: " ".join([token.lemma_ for token in nlp(x)])
    )
    return df

Und wenden diese anschließend auf unseren DataFrames an:

In [ ]:
train_df = enrich_lemmatized_text(train_df)
valid_df = enrich_lemmatized_text(valid_df)
test_df = enrich_lemmatized_text(test_df)

Nun können wir mit der Tf-Idf Vektorisierung beginnen, welche unsere Texte in ein numerisches Format umwandelt.

In [ ]:
vectorizer = TfidfVectorizer()

train_X = vectorizer.fit_transform(train_df["lemmatized_text"]).astype(np.float32)
valid_X = vectorizer.transform(valid_df["lemmatized_text"]).astype(np.float32)
test_X = vectorizer.transform(test_df["lemmatized_text"]).astype(np.float32)

Unsere Daten sind vorbereitet, wir können mit dem Bau unserer KI-Modelle beginnen. Wir trainieren:
- einen Entscheidungsbaum
- einen Random Forest
- eine logistische Regression
- ein künstliches neuronales Netzwerk

### Decision Tree (Entscheidungsbaum)

Ein Entscheidungsbaum ist als sehr einfaches Modell und wird oft als eines der ersten Modelle verwendet. Wir trainieren und geben verschiedene Optionen für die Hyperparameter `max_depth` und `min_samples_split`. Die Hyperparameter-Suche GridSearch wählt hierbei die optimalen Hyperparameter aus.

In [ ]:
tree_clf = DecisionTreeClassifier()
tree_params = {
    'max_depth': list(range(10, 101, 20)) + [None],
    'min_samples_split': [2, 5]
}
tree_search = GridSearchCV(tree_clf, tree_params)
tree_search.fit(train_X, train_df["label"])
best_tree_clf = tree_search.best_estimator_

Anschließend wollen wir unser Modell auf den Validierungsdaten prüfen. Da wir das wiederholt machen werden, bauen wir wieder eine Hilfsfunktion:

In [ ]:
def evaluate_clf(valid_X, labels, clf):
    predictions = clf.predict(valid_X)
    report = classification_report(labels, predictions)
    print(report)

Anschließend führen wir diese aus und können die Ergebnisse prüfen:

In [ ]:
evaluate_clf(valid_X, valid_df["label"], best_tree_clf)

### Random Forest

Als nächstes Modell eignet sich ein Random Forest, welcher eine Ensemble-Technik darstellt. Im Wesentlichen werden hier mehrere Entscheidungsbäume trainiert, und deren Ergebnis vereint. Wir haben zusätzlich zu den vorherigen Hyperparametern noch `n_estimators` zu wählen.

In [ ]:
forest_clf = RandomForestClassifier()
forest_params = {
    'n_estimators': list(range(10, 101, 20)),
    'max_depth': list(range(10, 101, 20)) + [None],
    'min_samples_split': [2, 5]
}
forest_search = GridSearchCV(forest_clf, forest_params)
forest_search.fit(train_X, train_df["label"])
best_forest_clf = forest_search.best_estimator_

Wir evaluieren wieder das Ergebnis:

In [ ]:
evaluate_clf(valid_X, valid_df["label"], best_forest_clf)

### Logistic Regression

Nun bauen wir eine logistische Regression. Diese ist ebenfalls sehr simpel, erweist sich oftmals aber als sehr gutes Modell. Wir haben vier Hyperparameter:

In [ ]:
lr_clf = LogisticRegression()
lr_params = {
    'penalty': ['l1', 'l2'],
    'max_iter': [100],
    'C': np.logspace(-4, 4, 20),
    'solver': ['liblinear']
}
lr_search = GridSearchCV(lr_clf, lr_params)
lr_search.fit(train_X, train_df["label"])
best_lr_clf = lr_search.best_estimator_

Und wir evaluieren erneut:

In [ ]:
evaluate_clf(valid_X, valid_df["label"], best_lr_clf)

### Feedforward Neural Network

Zuletzt wollen wir noch ein künstliches neuronales Netz bauen. Dafür haben wir eine Architektur in der Datei `ffnn.py` gewählt, welche wir zuvor importiert haben und nun mit der Bibliothek skorch einfach anwenden können. Wir wählen hier direkt die Hyperparameter ohne GridSearch:

In [ ]:
neural_net = NeuralNetClassifier(
    module=NeuralNetModule,
    module__num_inputs = len(vectorizer.vocabulary_),
    max_epochs=10,
    optimizer=torch.optim.Adam,
    iterator_train__shuffle=True,
    verbose=0
)
neural_net.fit(train_X, train_df['label'])

Nun prüfen wir das Ergebnis:

In [ ]:
evaluate_clf(valid_X, valid_df["label"], neural_net)

Unser bestes Modell auf den Validierungsdaten ist die logistische Regression, daher wählen wir diese als unser finales Modell.

### Test unseres Modells

Wir haben die Validierungsdaten gewählt, um unser bestes Modell auszuwählen. Es kann aber sein, dass unser Modell auf den Validierungsdaten nur zufällig gute Prognosen erzeugt hatte oder wir uns zu sehr durch die Hyperparameter-Optimierung auf unser Validierungs-Daten "überangepasst" haben. Daher evaluieren wir auf einem noch vollkommen "neuen" Teil der Daten, unseren Testdaten:

In [ ]:
evaluate_clf(test_X, test_df["label"], best_lr_clf)

Die Ergebnisse sind sogar tatsächlich etwas besser, als auf unseren Validierungsdaten.

Wir könnten an dieser Stelle noch zahlreiche weitere Untersuchungen machen. Wir könnten beispielsweise untersuchen, ob es Unterschiede in der Prognosequalität abhängig von der Textlänge gibt, oder ob unser Modell Verneinungen erkennt und korrekt interpretiert. Für dieses Projekt belassen wir es aber bei den Ergebnissen.